<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Introduction" data-toc-modified-id="Introduction-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Introduction</a></span></li><li><span><a href="#Environment" data-toc-modified-id="Environment-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Environment</a></span><ul class="toc-item"><li><span><a href="#Librairies" data-toc-modified-id="Librairies-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Librairies</a></span></li><li><span><a href="#Data-Loading" data-toc-modified-id="Data-Loading-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Data Loading</a></span></li><li><span><a href="#Functions" data-toc-modified-id="Functions-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Functions</a></span></li><li><span><a href="#Paths" data-toc-modified-id="Paths-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Paths</a></span></li></ul></li><li><span><a href="#Preprocessing-functions" data-toc-modified-id="Preprocessing-functions-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Preprocessing functions</a></span></li><li><span><a href="#Scraping" data-toc-modified-id="Scraping-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Scraping</a></span><ul class="toc-item"><li><span><a href="#Airliners" data-toc-modified-id="Airliners-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Airliners</a></span></li><li><span><a href="#Google-images" data-toc-modified-id="Google-images-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Google images</a></span></li></ul></li><li><span><a href="#Statistics-on-data" data-toc-modified-id="Statistics-on-data-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Statistics on data</a></span><ul class="toc-item"><li><span><a href="#Seatguru" data-toc-modified-id="Seatguru-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Seatguru</a></span></li><li><span><a href="#Hackathon" data-toc-modified-id="Hackathon-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>Hackathon</a></span></li></ul></li><li><span><a href="#Models" data-toc-modified-id="Models-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Models</a></span></li></ul></div>

<h1>Groupe 7 : Images sociales<span class="tocSkip"></span>
    
 Résultats
 
 Authors : All members
 

# Introduction

In [ ]:
Expliquer les choix : extérieur : utiliser directement les modèles
Création des catégories extérieur vu de l'intérieur et meal        
Transfert learning :

# Environment



## Librairies

In [ ]:
!pip install tensorflow==2.0.0
!pip install keras==2.3.1

In [1]:
import os

import time
from datetime import datetime, timedelta


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import imread


import csv

# Scraping

import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import urllib
import urllib.request
import requests
import errno
import io
from PIL import Image


import tensorflow
print(tensorflow.__version__)

import keras
print(keras.__version__)


2.0.0
2.3.1


Using TensorFlow backend.


## Data Loading

In [8]:
# Paths
project_path = './../'
hack_path = project_path + 'InputsHackathon/'
seatguru_path = project_path + 'Interpromo2020/All Data/ANALYSE IMAGE/IMG SEATGURU/'
insta_path = project_path + 'Interpromo2020/All Data/ANALYSE IMAGE/IMG INSTAGRAM/'
scrap_path = project_path + 'Scraping/'
airliners_path = scrap_path + 'Airliners/data/'
google_path = scrap_path + 'Google_img/'
stats_path = project_path + 'ImagesStats/'


## Functions

In [3]:
%run g7_functions_for_models.ipynb
%run preprocessing_functions.ipynb

ERROR:root:File `'g7_functions_for_models.ipynb.py'` not found.
ERROR:root:File `'preprocessing_functions.ipynb.py'` not found.


# Scraping

## Airliners

In [ ]:
### Scraping the data

In [ ]:
def get_photos_name_page(link):
    req = requests.get(link)
    soup = BeautifulSoup(req.text, "lxml")
    AirCrafts = soup.find_all("div", {"class":"ps-v2-results-row"})
    list_DF = []
    for air in AirCrafts:
        try :
            photo = air.find_all("img")[0].attrs['src']
            details_cell = air.find_all("div", {"class" : "ps-v2-results-col ps-v2-results-col-aircraft"})[0]
            details_cell = details_cell.find_all("div", {'class' : "ps-v2-results-col-content"})[0]
            details_cell = details_cell.find_all("div", {"class":"ps-v2-results-col-content-primary"})[0]
            name_cell = details_cell.find_all("div", {"class":"ps-v2-results-display-detail-no-wrapping"})[1]
            name = name_cell.a.text.strip().replace('/', '-')
            if name.split()[0] in ['Airbus', 'Boeing']:
                list_DF.append([photo, name.split()[0], name.split()[1].split('-')[0], name])
        except :
            pass
    DF = pd.DataFrame(list_DF, columns = ['Photo', 'Company', 'Aircraft_type', 'Designation'])
    return(DF)

def create_directory (path):
    os.makedirs(path + 'Airbus', exist_ok = True)   
    os.makedirs(path + 'Boeing', exist_ok = True)


def create_df_manufacturer(a,b,manuf_name):
    l_df = []
    dic_code_manuf = {'Airbus' : '2', 'Boeing' : '7'}
    for i in range(a,b):
        print(i)
        link = 'https://www.airliners.net/search?aircraftManufacturer=' + dic_code_manuf[manuf_name]+ \
               '&photoCategory=23&sortBy=dateAccepted&sortOrder=desc&perPage=36&display=detail&page=' + str(i)
        l_df.append(get_photos_name_page(link))
    df = pd.concat(l_df, ignore_index = True)
    return df

In [ ]:
create_directory(airliners_path)

In [ ]:
# Scraping for Airbus

df_airbus_test = create_df_manufacturer(1,100,'Airbus')
df_airbus_test.to_csv(airliners_path + 'Airbus/' + 'Airbus_test.csv', sep = ';')
df_airbus_test.head()

In [ ]:
df_airbus_airliners = create_df_manufacturer(1,17452,'Airbus')
df_airbus_airliners.to_csv(airliners_path + 'Airbus/Airbus_Airliners.csv', sep = ';')
len(df_airbus_airliners)

In [ ]:
df_boeing_airliners = create_df_manufacturer(1,28482,'Boeing')
df_boeing_airliners.to_csv(airliners_path + 'Boeing/Boeing_Airliners.csv', sep = ';')
len(df_airbus_airliners)

In [ ]:
## Creating the images from the dataframe

In [ ]:
df_airbus = pd.read_csv(airliners_path + 'Airbus/Airbus_Airliners.csv', sep = ';', index_col = 'Unnamed: 0')
df_boeing = pd.read_csv(airliners_path + 'Boeing/Boeing_Airliners.csv', sep = ';', index_col = 'Unnamed: 0')

In [ ]:
dic_type = {'Airbus' : ['A220', 'A300B4', 'A310', 'A318', 'A319', 'A320', 'A321', 'A330', 'A340', 'A350', 'A380'],
            'Boeing' : ['717', '727', '737', '747', '757', '767', '777', '787']}

In [ ]:
# Creation and storage of the images

In [ ]:
def create_df_passengers(df, company, dic_type):
    """Create a new dataframe including only rows matching with commercial aircraft and adding a column
    similar to the index.
    
    Parameters : 
        df (type dataframe): dataframe on which the selection must be proceed
        company (type string): company name ('Airbus' or 'Boeing')
        dic_type (type dictionary): keys are companies names and values are lists of the commercial aircraft 
        of the company 
    
    Out :
        df_passengers (type dataframe): modified dataframe  
    """
        
    df_passengers = df[df['Aircraft_type'].isin(dic_type[company])]
    return df_passengers.assign(Number = df_passengers.index)



def get_from_link_and_write(df, company, path):
    
    for el in df['Aircraft_type'].unique().tolist():
        df_type = df[df['Aircraft_type']== el]
        fold_name = path + company + '/' + el
        os.makedirs(fold_name, exist_ok = True)
        my_rows = zip(df_type['Number'], df_type['Photo'])
        for (num, photo) in my_rows:
            img_data = requests.get(photo).content
            try :
                i = Image.open(io.BytesIO(img_data))
                i.save(fold_name + '/' + company + '_' + str(num) + '_' + el + '.png')
            except :
                print(el, num)
                pass
        
def get_number_pic_per_type(df, nb, a):
    # df : dataframe with only aircraft types of interest
    # nb : number of pictures to create for ech aircraft type
    ix = []
    for el in df['Aircraft_type'].unique().tolist():
        df_el = df[df['Aircraft_type']== el].copy()
        df_el.reset_index(drop = True, inplace = True)
        df_el = df_el.loc[a : a + min(nb-1, len(df_el))]
        ix.extend(df_el['Number'].tolist())
    df_nb = df[df['Number'].isin(ix)]
    return df_nb

In [ ]:
df_airbus_passengers = create_df_passengers(df_airbus, 'Airbus', dic_type)
df_boeing_passengers = create_df_passengers(df_boeing, 'Boeing', dic_type)


In [ ]:
for el in dic_type['Airbus'] : 
    print(el, len(df_airbus[df_airbus['Aircraft_type']== el]))
    
print('\n')

for el in dic_type['Boeing'] : 
    print(el, len(df_boeing[df_boeing['Aircraft_type']== el]))

In [ ]:
df_airbus_100 = get_number_pic_per_type(df_airbus_passengers, 100, 0)
print(len(df_airbus_100))

In [ ]:
df_airbus_100.head()

In [ ]:
start=time.time()
get_from_link_and_write(df_airbus_100, 'Airbus', path)
print(time.time()-start)
# 125s needed to write 1100 pictures

In [ ]:
df_airbus_limited = get_number_pic_per_type(df_airbus_passengers, 1000, 0)
get_from_link_and_write(df_airbus_limited, 'Airbus', path)

In [ ]:
df_boeing_limited = get_number_pic_per_type(df_boeing_passengers, 1000, 0)
get_from_link_and_write(df_boeing_limited, 'Boeing', path)

## Google images

In [ ]:
driver = webdriver.Chrome("chromedriver.exe")
os.makedirs(google_path, exist_ok = True)


In [ ]:
dic_df_types = {}


In [ ]:
def google_img_by_aircraft_type(aircraft_type, dic_df_types):
    
    """ Update a dictionary with a dataframe giving links to the pictures found on the first page of Google Images
    for a given request.
    
    Parameters :
        aircraft_type (type string) : request for Google Images. Ex : 'Airbus A380'.
        dic_df_types (type dict) : the keys are aircraft_type and the values are dataframes.
    """ 
    
    # Use of &tbs=sur:fc in the link to download only free_to_use images : 
    link = 'https://www.google.com/search?q=' + aircraft_type + "&tbm=isch&tbs=sur:fc"
    df_images = pd.DataFrame(columns=["link", "request", "source", "source_2"])
    driver.get(link)
    time.sleep(5)

    blocks_images = driver.find_elements_by_xpath("//div[contains(@class,'rg_bx rg_di rg_el ivg-i')]")
    for bi in blocks_images:

        site = bi.find_elements_by_xpath(".//span")
        if site : site = site[0].get_attribute("innerHTML")

        source_1 = bi.find_elements_by_xpath(".//img")
        if source_1 : source_1 = source_1[0].get_attribute("data-src")

        source_2 = bi.find_elements_by_xpath("./a")
        if source_2 : source_2 = source_2[0].get_attribute("href")

        row = pd.Series([site, aircraft_type, source_1, source_2], index=df_images.columns)
        df_images = df_images.append(row, ignore_index=True, sort=False)
        
    dic_df_types[aircraft_type] = df_images
    
    
    
def get_from_source_and_write(request, dic_df_types, path):
    
    print(request)
    df = dic_df_types[request]
    print(len(df))
    df = df[df['source'].notnull()]
    df.reset_index(inplace = True)
    df = df.assign(Number = df.index)
    print(len(df))
    request = request.split()    
    fold_name = path + request[0] + '/' + request[1]
    os.makedirs(fold_name, exist_ok = True)
    my_rows = zip(df['Number'], df['source'])
    for (num, source) in my_rows:
        img_data = requests.get(source).content
        try :
            i = Image.open(io.BytesIO(img_data))
            i.save(fold_name + '/' + request[0] + '_' + request[1] + '_' + str(num) + '.png')
        except :
            pass


In [ ]:
for key in dic_type.keys():
    print(key)
    for aircraft in dic_type[key]:
        print(aircraft)
        google_img_by_aircraft_type(key + ' ' + aircraft, dic_df_types)
    

In [ ]:
dic_df_types['Boeing 737'].head()

In [ ]:
# Test with an aircraft type 

key = 'Boeing'
#['717', '727', '737', '747', '757', '767', '777', '787']
aircraft = '787'
get_from_source_and_write(key + ' ' + aircraft, dic_df_types, path)   

In [ ]:
for key in dic_type.keys():
    for aircraft in dic_type[key]:
        get_from_source_and_write(key + ' ' + aircraft, dic_df_types, path)   

In [ ]:
### Save dataframes

In [ ]:
def saver(dic_df_types):
    for key, val in dic_df_types.items():
        val.to_csv(google_path + "Google_{}.csv".format(str(key)), sep = ';')

    with open(google_path + "keys.txt", "w") as f: #saving keys to file
        f.write(str(list(dic_df_types.keys())))

def loader():
    """Reading data from keys"""
    with open(google_path + "keys.txt", "r") as f:
        keys = eval(f.read())

    dic = {}    
    for key in keys:
        dic[key] = pd.read_csv(google_path + "Google_{}.csv".format(str(key)), sep = ';')
    return dic



In [ ]:
saver(dic_df_types)
dic = loader()
dic

# Statistics on data 

## Seatguru

In [9]:
img_list = os.listdir(seatguru_path)
nb_images = len(img_list)

FileNotFoundError: [Errno 2] No such file or directory: './../Interpromo2020/All Data/ANALYSE IMAGE/IMG SEATGURU/'

In [ ]:
# Create Dataframe

In [ ]:
# List of all matrices (images)
imgs = list()
for i in range(nb_images):
    img = imread(path + img_list[i])
    imgs.append(img)

# Init the dataframe that will contain all basic info about images
imgs_df = pd.DataFrame(columns = ['name', 'format', 'height', 'width',
                                  'height_to_width', 'ncol', 'aircraft_manufacturer', 'aircraft_type'])
imgs_df.name = img_list

In [ ]:
def get_imgs_formats(imgs: list, imgs_df: pd.DataFrame) -> pd.DataFrame:
    """Fills the DataFrame with pictures info (format and dimensions).
    
    Parameters:
    imgs: list of arrays representing images
    imgs_df: empty DataFrame with at least a 'name' column

    Out:
    imgs_df: DataFrame with 4 new columns: 'format', 'height', 'width',
             'height_to_width', 'ncol', and one line per image

    """

    # Formats
    imgs_df['format'] = imgs_df.name.apply(lambda x: x.split('.')[1])

    # Shapes
    heights = [imgs[k].shape[0] if len(imgs[k].shape) != 0 else 0 for k in range(len(imgs))]
    widths = [imgs[k].shape[1] if len(imgs[k].shape) != 0 else 0 for k in range(len(imgs))]
    ncols = [imgs[k].shape[2] if len(imgs[k].shape) != 0 else 0 for k in range(len(imgs))]

    imgs_df['height'] = heights
    imgs_df['width'] = widths
    imgs_df['height_to_width'] = [x / y if y != 0 else 0 for x, y in zip(imgs_df.height, imgs_df.width)]
    imgs_df['ncol'] = ncols

    return imgs_df

In [ ]:
# Apply function to fill in DataFrame
imgs_df = get_imgs_formats(imgs, imgs_df)

# Aircraft manufacturers
aircraft_manufacturers = ['Airbus' if 'Airbus' in imgs_df.name[k] 
                          else 'Boeing' if 'Boeing' in imgs_df.name[k]
                          else 'Other' for k in range(len(imgs_df))]

# Aircraft types
aircraft_types = [name.split('_')[name.split('_').index(aircraft_manufacturer) + 1].split('-')[0]
                  if aircraft_manufacturer in name.split('_') else ''
                  for name, aircraft_manufacturer in zip(imgs_df.name, aircraft_manufacturers)]

# Add missing As for Airbus aircrafts
aircraft_types = [aircraft_types[k] 
                  if ('A' in aircraft_types[k] or '7' in aircraft_types[k] or aircraft_types[k] == '')
                  else 'A' + aircraft_types[k] for k in range(len(aircraft_types))]

# Remove Airbus 'neos' because we don't need that much detail
aircraft_types = [aircraft_types[k].replace('neo', '') for k in range(len(aircraft_types))]

# Remove Ds
aircraft_types = [aircraft_types[k].replace('D', '') for k in range(len(aircraft_types))]

# Fill in dataframe columns
imgs_df.aircraft_manufacturer = aircraft_manufacturers
imgs_df.aircraft_type = aircraft_types

In [ ]:
imgs_df.head()

In [ ]:
# Save as csv
path_tr =  './../CSV_annotate/SEATGURU/'
os.makedirs(path_tr, exist_ok=True)
imgs_df.to_csv(path_or_buf=path_tr + 'g7_SEATGURU.csv', sep=';', encoding='utf-8', index=False)

In [ ]:
### Descriptive statistics¶


In [ ]:
print(f'{len(np.unique(imgs_df.format))} unique image format(s).')
print(f'{len(np.unique(imgs_df.ncol))} unique ncol(s).')
print(f'{len(np.unique(imgs_df.height_to_width))} unique height_to_width(s).')
print(f'{len(np.unique(imgs_df.aircraft_type))} unique aircraft type(s).')

In [ ]:
print(f'{len(np.arange(nb_images)[imgs_df.aircraft_manufacturer == "Airbus"])} Airbus labelled images.')
print(f'{len(np.arange(nb_images)[imgs_df.aircraft_manufacturer == "Boeing"])} Boeing labelled images.')
print(f'{len(np.arange(nb_images)[imgs_df.aircraft_manufacturer == "Other"])} others.')

In [ ]:
def get_stats(imgs_df: pd.DataFrame, col: pd.Series, col_name: str):

    print(f'Max {col_name}: {np.max(col)}')
    print(f'Median {col_name}: {np.median(col)}')
    print(f'Min {col_name}: {np.min(col)}')
    print('')

In [ ]:
# Dimensions
get_stats(imgs_df=imgs_df, col=imgs_df.height_to_width,
          col_name='height_to_width')

get_stats(imgs_df=imgs_df, col=imgs_df.height,
          col_name='height')

get_stats(imgs_df=imgs_df, col=imgs_df.width,
          col_name='width')

In [ ]:
# Number of images per aircraft type
pd.DataFrame(
    pd.pivot_table(imgs_df,
    index=['aircraft_manufacturer', 'aircraft_type'],
    aggfunc='count').format.sort_values(ascending=False))

In [9]:
## Labellisation added manually : add SEATGURU annotate

## Hackathon

## Instagram

In [ ]:
# General path 
directory = './Interpromo2020/All Data/ANALYSE IMAGE/INSTAGRAM/'

In [ ]:
def dim_insta(i, num_dim, end_path):
    """
       
    Parameters:
        i (type: int): index
        num_dim (type: int): index of the shape of the picture
        end_path (type: string): work directory
        
    Out:
        shape of the picture
    
    """
    
    try : 
        img = imread(end_path + '/' + str(i) + '.jpg')
        return int(img.shape[num_dim])
    except :
        print(i)
        return 0
        pass

In [ ]:
def data_insta(end_path): 
    """
    Return CSV file which contains shapes of all the pictures from the folder
    
    Parameters:
        end_path (type: string): work directory
        
    
    
    """
    
    # Step 1: complete the path
    directory_complete = directory + end_path
    
    # Step 2: recover the name of each picture
    list_pic = os.listdir(directory_complete)
    list_pic = [int(el.split('.')[0]) for el in list_pic if el.split('.')[1] == 'jpg']
    list_pic = sorted(list_pic)
    
    # Step 3: creation of a dataframe
    df = pd.DataFrame(index = list_pic , columns = ['height', 'width'])
    
    # Step 4: duplicate the index column
    df = df.assign(Number = df.index)
    
    # Step 5: application of the dim function on columns height and width
    df['height'] = df['Number'].apply(lambda x : dim_insta(x, 0,directory_complete))
    df['width'] = df['Number'].apply(lambda x : dim_insta(x, 1,directory_complete))
    
    
    df = df.drop(['Number'], axis = 1)
    
    # Step : creation of the csv
    df.to_csv(directory_complete + '/dataframe_' + end_path + '.csv', sep = ';', index = True)

In [ ]:
df_insta_airbus = data_insta('airbus')

In [ ]:
df_insta_int = data_insta('aircraftinterior')

In [7]:
df_insta_aircraft_seat = data_insta('aircraftseat')

NameError: name 'data_insta' is not defined

In [8]:
df_insta_aircraft_seat = data_insta('aircraftseat')

NameError: name 'data_insta' is not defined

# Models

Nous avons commencé par essayer de comprendre la problématique pour définir le nombre de modèles à entraîner.
Pour l'intérieur et pour l'extérieur, nous pensions commencer par construire un modèle qui prédit le constructeur puis un modèle qui prédit le modèle d'avion.

Vu que nous avons obtenu des résultats satisfaisants pour les extérieurs d'avion en commençant par détecter en simultanés les modèles d'avion d'Airbus et de Boeing, nous n'avons pas utilisé pour l'extérieur.


Nous avons commencé par tester des CNN (Donner des scores)
Face aux résultats décevants, nous avons décidé de tenter le transfert learning.
Nous avons donc repris les CNN en ajoutant au départ un réseau pré-entrainé.

Or, pour les trois réseaux qui doivent prédire les intérieurs nous nous sommes heurtés au même problème, : l'accuracy au niveau de l'entrainement s'approche de 1, alors que l'acurracy au niveau de la validation  plafonne très rapidement. (Préciser la valeur pour chacun des modèles ? ) Vu qu'il y a peu d'images, le réseau apprend par coeur et ne peut pas généraliser.

La structure du modèle employée est la suivante : 

Pour les intérieurs, nous avons également essayé plusieurs combinaisons pour détecter les modèles et le constructeur : 
Pour ce qui concerne Airbus : utiliser, Seatguru, utiliser les données du Hackhaton ou bien utiliser les deux combinées pour obtenir un plus grand nombre d'images.

En effet, les données du Hackathon étaient plus nombreuses que les données de SeatGuru pour chaque type de modèle.
Toutefois, les images sont différentes de celles qu'on peut trouver dans les réseaux sociaux puisqu'il n'y a jamais de personnes présentes. 

Face aux résultats décevants, nous avons cherché à pratiquer la data augmentation.(Combien d'images supplémentaires générées ?
Les résultats n'étaient pas plus concluants.

Le manque de données fournies, combinées avec le fait qu'au niveau du Hackhaton les données fournies sont différentes des données de test, ne permettent pas d'entraîner convenablement le modèle.

Un plus grand nombre de données pourraient être une solution pour améliorer les performances.

## Models creation

### Viewpoint 

### Exterior : Aircraft_type

### Interior : Manufacturer

### Interiors : Airbus aircrafts

### Interiors : Boeing aircrafts

### Airbus exteriors

## Models evaluation

In [ ]:
# Results

In [ ]:
### Pipeline

In [ ]:
### Statistics

In [ ]:
### Instagram

# Conclusion